In [1]:
## Import dependencies
import sqlalchemy
from sqlalchemy import create_engine
import sys
sys.path.append('../')
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import sqlite3


In [2]:
## load data from csv file to pandas dataframe
MMSI_2016 = pd.read_csv("data/Movilidad_Social_datos.csv")
MMSI_2016.head()

C:\Users\Jorge\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3049: DtypeWarning: Columns (27,28,30,31,32,33,34,35,49,50,51,52,53,54,55,56,57,58,59,184,185,211,236) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Folioviv,Foliohog,Id_pobla,ubica_geo_ENH,ent_ENH,tam_loc_ENH,est_socio_ENH,est_dis_ENH,upm_ENH,P1_1,...,cisterna_ENH,pileta_ENH,calent_sol_ENH,calent_gas_ENH,medidor_luz_ENH,bomba_agua_ENH,tanque_gas_ENH,aire_acond_ENH,calefacc_ENH,chimenea_ENH
0,100018005,1,1,10010001,1,1,3,5,1,1,...,1,2,1,1,1,1,1,2,2,2
1,100018006,1,3,10010001,1,1,3,5,1,1,...,1,2,1,1,1,1,1,2,2,2
2,100018036,1,3,10010001,1,1,3,5,1,1,...,1,2,1,1,1,1,1,2,2,2
3,100018068,1,1,10010001,1,1,3,5,1,1,...,1,2,2,1,1,1,1,2,2,2
4,100018092,1,3,10010001,1,1,3,5,1,2,...,1,2,2,1,1,1,2,2,2,2


In [3]:
## Filtering the columns we need from the main dataset
MMSI_2016_summ = MMSI_2016.loc[:,["ent_ENH","est_socio_ENH","P1_1","P1_2","P2_1EST_D","P10_1","P10_2","P10_3","P12_1","P12_2","Factor_Per",]]
MMSI_2016_summ.head()

,ent_ENH,est_socio_ENH,P1_1,P1_2,P2_1EST_D,P10_1,P10_2,P10_3,P12_1,P12_2,Factor_Per
0,1,3,1,58,1,2,8,3,6,4,1718
1,1,3,1,49,7,2,8,4,5,5,859
2,1,3,1,31,1,2,8,4,7,7,429
3,1,3,1,56,1,2,8,3,5,3,859
4,1,3,2,52,1,2,7,3,5,4,329


In [4]:
## Counting the number of rows on each column
MMSI_2016_summ.count()

ent_ENH          25634
est_socio_ENH    25634
P1_1             25634
P1_2             25634
P2_1EST_D        25634
P10_1            25634
P10_2            25634
P10_3            25634
P12_1            25634
P12_2            25634
Factor_Per       25634
dtype: int64

In [5]:
## Renaming the columns
MMSI_2016_summ=MMSI_2016_summ.rename(columns={"ent_ENH":"Estado","est_socio_ENH":"Estrato_Socioeconómico",
                               "P1_1":"Sexo","P1_2":"Edad",
                               "P2_1EST_D":"Edo_14_años",
                              "P10_1":"Lengua_Indígena","P10_2":"Color_Piel","P10_3":"Origen_Racial",
                               "P12_1":"Opinión_Situación_Económica","P12_2":"Perspectiva_SE_14_años",
                               "Factor_Per":"Factor_Expansión"})
MMSI_2016_summ.head()

,Estado,Estrato_Socioeconómico,Sexo,Edad,Edo_14_años,Lengua_Indígena,Color_Piel,Origen_Racial,Opinión_Situación_Económica,Perspectiva_SE_14_años,Factor_Expansión
0,1,3,1,58,1,2,8,3,6,4,1718
1,1,3,1,49,7,2,8,4,5,5,859
2,1,3,1,31,1,2,8,4,7,7,429
3,1,3,1,56,1,2,8,3,5,3,859
4,1,3,2,52,1,2,7,3,5,4,329


In [6]:
## Using a dictionary to know the real result of each column
MMSI_2016_summ["Estrato_Socioeconómico"]=MMSI_2016_summ["Estrato_Socioeconómico"].replace({1:"Bajo",
                                                                                           2:"Medio bajo",
                                                                                           3:"Medio alto",
                                                                                           4:"Alto"})
MMSI_2016_summ["Sexo"]=MMSI_2016_summ["Sexo"].replace({1:"Hombre",
                                                       2:"Mujer"})
MMSI_2016_summ["Lengua_Indígena"]=MMSI_2016_summ["Lengua_Indígena"].replace({1:"Sí",
                                                                             2:"No"})
MMSI_2016_summ["Origen_Racial"]=MMSI_2016_summ["Origen_Racial"].replace({1:"Negra o Mulata",
                                                                         2:"Indígena",
                                                                         3:"Mestiza",
                                                                        4:"Blanca",
                                                                        5:"Otra Raza",
                                                                        9:"No sabe"})
MMSI_2016_summ["Estado"]=MMSI_2016_summ["Estado"].replace({32:"Zac.",31:"Yuc.",30:"Ver.",29:"Tlax.", 28:"Tamps.",
                                                           27:"Tab.",26:"Son.",25:"Sin.", 24:"SLP",23:"Q. Roo",
                                                           22:"Qro.", 21:"Pue.",20:"Oax.",19:"NL",18:"Nay.", 17:"Mor.",
                                                           16:"Mich.", 15:"Mex.",14:"Jal.",13:"Hgo.",12:"Gro.",11:"Gto.",
                                                           10:"Dgo.",9:"CDMX",8:"Chih.",7:"Chis.",6:"Col.",5:"Coah.",
                                                           4:"Camp.",3:"BCS",2:"BC",1:"Ags."})
MMSI_2016_summ["Edo_14_años"]=MMSI_2016_summ["Edo_14_años"].replace({32:"Zac.",31:"Yuc.",30:"Ver.",29:"Tlax.", 28:"Tamps.",
                                                           27:"Tab.",26:"Son.",25:"Sin.", 24:"SLP",23:"Q. Roo",
                                                           22:"Qro.", 21:"Pue.",20:"Oax.",19:"NL",18:"Nay.", 17:"Mor.",
                                                           16:"Mich.", 15:"Mex.",14:"Jal.",13:"Hgo.",12:"Gro.",11:"Gto.",
                                                           10:"Dgo.",9:"CDMX",8:"Chih.",7:"Chis.",6:"Col.",5:"Coah.",
                                                           4:"Camp.",3:"BCS",2:"BC",1:"Ags."})

In [7]:
## Showing the first 5 rows of the dataset
MMSI_2016_summ.head()

,Estado,Estrato_Socioeconómico,Sexo,Edad,Edo_14_años,Lengua_Indígena,Color_Piel,Origen_Racial,Opinión_Situación_Económica,Perspectiva_SE_14_años,Factor_Expansión
0,Ags.,Medio alto,Hombre,58,Ags.,No,8,Mestiza,6,4,1718
1,Ags.,Medio alto,Hombre,49,Chis.,No,8,Blanca,5,5,859
2,Ags.,Medio alto,Hombre,31,Ags.,No,8,Blanca,7,7,429
3,Ags.,Medio alto,Hombre,56,Ags.,No,8,Mestiza,5,3,859
4,Ags.,Medio alto,Mujer,52,Ags.,No,7,Mestiza,5,4,329


In [8]:
## Making a "group by" by state 
MMSI_2016_nacional_prom=MMSI_2016_summ.groupby(["Estado"], as_index=False).mean()
MMSI_2016_nacional_sum=MMSI_2016_summ.groupby(["Estado"],as_index=False).sum()
MMSI_2016_nacional=MMSI_2016_nacional_prom.loc[:,["Estado","Opinión_Situación_Económica","Perspectiva_SE_14_años"]]
MMSI_2016_nacional_sum=MMSI_2016_nacional_sum.loc[:,["Estado","Factor_Expansión"]]

In [9]:
## Meging the results in a single dataset
MMSI_2016_nacional=MMSI_2016_nacional.merge(MMSI_2016_nacional_sum, on="Estado")
MMSI_2016_nacional["Cambio_Perspectiva"]=MMSI_2016_nacional["Opinión_Situación_Económica"] - MMSI_2016_nacional["Perspectiva_SE_14_años"]
MMSI_2016_nacional=MMSI_2016_nacional.sort_values(by=["Cambio_Perspectiva"])
MMSI_2016_nacional.head()

,Estado,Opinión_Situación_Económica,Perspectiva_SE_14_años,Factor_Expansión,Cambio_Perspectiva
25,Son.,5.257362,4.891165,1554799,0.366197
3,CDMX,5.356322,4.959132,5097738,0.397190
1,BC,5.752355,5.340511,1874447,0.411844
18,Nay.,4.377070,3.963057,617941,0.414013
9,Dgo.,4.930468,4.505689,839716,0.424779


In [10]:
## Creating a dataset only with the variable "gender"
MMSI_2016_sex=MMSI_2016_summ.groupby(["Sexo","Perspectiva_SE_14_años","Opinión_Situación_Económica"],as_index=False).sum()
MMSI_2016_sex=MMSI_2016_sex.loc[:,["Sexo","Perspectiva_SE_14_años","Opinión_Situación_Económica","Factor_Expansión"]]
MMSI_2016_sex["Demográfico"]="Sexo"
MMSI_2016_sex=MMSI_2016_sex.rename(columns={"Sexo":"Varible"})
MMSI_2016_sex.head()

,Varible,Perspectiva_SE_14_años,Opinión_Situación_Económica,Factor_Expansión,Demográfico
0,Hombre,1,1,1289967,Sexo
1,Hombre,1,2,943751,Sexo
2,Hombre,1,3,1027508,Sexo
3,Hombre,1,4,549509,Sexo
4,Hombre,1,5,712872,Sexo


In [11]:
## Creating a dataset only with the variable "Racial Origin"
MMSI_2016_OriRacial=MMSI_2016_summ.groupby(["Origen_Racial","Perspectiva_SE_14_años","Opinión_Situación_Económica"],as_index=False).sum()
MMSI_2016_OriRacial=MMSI_2016_OriRacial.loc[:,["Origen_Racial","Perspectiva_SE_14_años","Opinión_Situación_Económica","Factor_Expansión"]]
MMSI_2016_OriRacial["Demográfico"]="Origen_Racial"
MMSI_2016_OriRacial=MMSI_2016_OriRacial.rename(columns={"Origen_Racial":"Varible"})
MMSI_2016_OriRacial.head()

,Varible,Perspectiva_SE_14_años,Opinión_Situación_Económica,Factor_Expansión,Demográfico
0,Blanca,1,1,281329,Origen_Racial
1,Blanca,1,2,164285,Origen_Racial
2,Blanca,1,3,181500,Origen_Racial
3,Blanca,1,4,128492,Origen_Racial
4,Blanca,1,5,141153,Origen_Racial


In [12]:
## Creating a dataset only with the variable "Skin Color"
MMSI_2016_Color=MMSI_2016_summ.groupby(["Color_Piel","Perspectiva_SE_14_años","Opinión_Situación_Económica"],as_index=False).sum()
MMSI_2016_Color=MMSI_2016_Color.loc[:,["Color_Piel","Perspectiva_SE_14_años","Opinión_Situación_Económica","Factor_Expansión"]]
MMSI_2016_Color["Demográfico"]="Color Piel"
MMSI_2016_Color=MMSI_2016_Color.rename(columns={"Color_Piel":"Varible"})
MMSI_2016_Color.head()

,Varible,Perspectiva_SE_14_años,Opinión_Situación_Económica,Factor_Expansión,Demográfico
0,1,1,1,25440,Color Piel
1,1,1,2,10147,Color Piel
2,1,1,3,4735,Color Piel
3,1,1,4,4819,Color Piel
4,1,1,5,2459,Color Piel


In [13]:
## Creating a dataset only with the variable "Indigenous languages"
MMSI_2016_LengInd=MMSI_2016_summ.groupby(["Lengua_Indígena","Perspectiva_SE_14_años","Opinión_Situación_Económica"],as_index=False).sum()
MMSI_2016_LengInd=MMSI_2016_LengInd.loc[:,["Lengua_Indígena","Perspectiva_SE_14_años","Opinión_Situación_Económica","Factor_Expansión"]]
MMSI_2016_LengInd["Demográfico"]="Lengua_Indígena"
MMSI_2016_LengInd=MMSI_2016_LengInd.rename(columns={"Lengua_Indígena":"Varible"})
MMSI_2016_LengInd.head()

,Varible,Perspectiva_SE_14_años,Opinión_Situación_Económica,Factor_Expansión,Demográfico
0,No,1,1,2156075,Lengua_Indígena
1,No,1,2,1492771,Lengua_Indígena
2,No,1,3,1968335,Lengua_Indígena
3,No,1,4,1134988,Lengua_Indígena
4,No,1,5,1579922,Lengua_Indígena


In [14]:
## Creating a dataset only with the variable "Socio-economic background"
MMSI_2016_Estrato=MMSI_2016_summ.groupby(["Estrato_Socioeconómico","Perspectiva_SE_14_años","Opinión_Situación_Económica"],as_index=False).sum()
MMSI_2016_Estrato=MMSI_2016_Estrato.loc[:,["Estrato_Socioeconómico","Perspectiva_SE_14_años","Opinión_Situación_Económica","Factor_Expansión"]]
MMSI_2016_Estrato["Demográfico"]="Estrato_Socioeconómico"
MMSI_2016_Estrato=MMSI_2016_Estrato.rename(columns={"Estrato_Socioeconómico":"Varible"})
MMSI_2016_Estrato.head()

,Varible,Perspectiva_SE_14_años,Opinión_Situación_Económica,Factor_Expansión,Demográfico
0,Alto,1,1,29329,Estrato_Socioeconómico
1,Alto,1,2,43558,Estrato_Socioeconómico
2,Alto,1,3,91262,Estrato_Socioeconómico
3,Alto,1,4,37250,Estrato_Socioeconómico
4,Alto,1,5,52831,Estrato_Socioeconómico


In [15]:
## Creating a dataset only with the variable "Generations"
Generación=["Millenials","X","Baby Boomers"]
Edades=[23,35,47,67]
MMSI_2016_Edad=MMSI_2016_summ.groupby(["Edad","Perspectiva_SE_14_años","Opinión_Situación_Económica"],as_index=False).sum()
MMSI_2016_Edad=MMSI_2016_Edad.loc[:,["Edad","Perspectiva_SE_14_años","Opinión_Situación_Económica","Factor_Expansión"]]
MMSI_2016_Edad["Generación"]=pd.cut(MMSI_2016_Edad["Edad"],Edades)
MMSI_2016_Rango=MMSI_2016_Edad.groupby(["Generación","Perspectiva_SE_14_años","Opinión_Situación_Económica"],as_index=False).sum()
MMSI_2016_Rango=MMSI_2016_Rango.loc[:,["Generación","Perspectiva_SE_14_años","Opinión_Situación_Económica","Factor_Expansión"]]
MMSI_2016_Gen=MMSI_2016_Rango
MMSI_2016_Gen["Generación"]=MMSI_2016_Gen["Generación"].astype(str)
MMSI_2016_Gen["Generación"]=MMSI_2016_Gen["Generación"].replace({"(23, 35]":"Millenials","(35, 47]":"X","(47, 67]":"Baby Boomers"})
MMSI_2016_Gen["Demográfico"]="Generación"
MMSI_2016_Gen=MMSI_2016_Gen.rename(columns={"Generación":"Varible"})
MMSI_2016_Gen.head()

,Varible,Perspectiva_SE_14_años,Opinión_Situación_Económica,Factor_Expansión,Demográfico
0,Millenials,1,1,665142.0,Generación
1,Millenials,1,2,372210.0,Generación
2,Millenials,1,3,543262.0,Generación
3,Millenials,1,4,252100.0,Generación
4,Millenials,1,5,256865.0,Generación


In [16]:
## Concating all the variable datasets in a signle dataset
MMSI_2016_Demográficos=pd.concat([MMSI_2016_sex,MMSI_2016_LengInd,MMSI_2016_OriRacial,MMSI_2016_Gen,MMSI_2016_Estrato,MMSI_2016_Color])
MMSI_2016_Demográficos["ID"]=np.arange(len(MMSI_2016_Demográficos))
MMSI_2016_Demográficos.head()

,Varible,Perspectiva_SE_14_años,Opinión_Situación_Económica,Factor_Expansión,Demográfico,ID
0,Hombre,1,1,1289967.0,Sexo,0
1,Hombre,1,2,943751.0,Sexo,1
2,Hombre,1,3,1027508.0,Sexo,2
3,Hombre,1,4,549509.0,Sexo,3
4,Hombre,1,5,712872.0,Sexo,4


In [17]:
## Showing all the variables
Demográficos=MMSI_2016_Demográficos["Demográfico"].unique()
Demográficos

array(['Sexo', 'Lengua_Indígena', 'Origen_Racial', 'Generación',
       'Estrato_Socioeconómico', 'Color Piel'], dtype=object)

In [18]:
## Exporting all the dataset to a csv file
MMSI_2016_sex.to_csv("data/csv/MMSI Sexo.csv")
MMSI_2016_LengInd.to_csv("data/csv/MMSI Lengua Indígena.csv")
MMSI_2016_OriRacial.to_csv("data/csv/MMSI Origen Racial.csv")
MMSI_2016_Rango.to_csv("data/csv/MMSI Rango de Edad.csv")
MMSI_2016_Estrato.to_csv("data/csv/MMSI Estrato Socioeconómico.csv")
MMSI_2016_Color.to_csv("data/csv/MMSI Color.csv")
MMSI_2016_Demográficos.to_csv("data/csv/MMSI Demográficos.csv")

In [19]:
## Creating a connection to sqlite
conn = sqlite3.connect('MMSI_2016.sqlite')

c = conn.cursor()

## Loading the dataset to a table in sqlite
MMSI_2016_Demográficos.to_sql('MMSI_2016_Demográficos', conn, if_exists='replace',index=False)
pd.read_sql('select * from MMSI_2016_Demográficos', conn) 

,Varible,Perspectiva_SE_14_años,Opinión_Situación_Económica,Factor_Expansión,Demográfico,ID
0,Hombre,1,1,1289967.0,Sexo,0
1,Hombre,1,2,943751.0,Sexo,1
2,Hombre,1,3,1027508.0,Sexo,2
3,Hombre,1,4,549509.0,Sexo,3
4,Hombre,1,5,712872.0,Sexo,4
5,Hombre,1,6,196358.0,Sexo,5
6,Hombre,1,7,65738.0,Sexo,6
7,Hombre,1,8,27007.0,Sexo,7
8,Hombre,1,9,8212.0,Sexo,8
9,Hombre,1,10,39238.0,Sexo,9
